[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DoranLyong/Awesome-Tensor-Architecture/blob/main/pytorch_reference/simple_reference/06_PyTorch_Acceleration_and_Optimization/03_Hyperparameter_Tuning.ipynb)

# Model Optimization


In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torchvision.models import vgg16

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
